In [1]:
!mkdir data/ 
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar -O data/ILSVRC2012_img_val.tar
!tar -xf data/ILSVRC2012_img_val.tar -C data/
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/imagenet_classes.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/ILSVRC2012_validation_ground_truth.txt

--2022-05-01 17:15:33--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘data/ILSVRC2012_img_val.tar’

data/ILSVRC2012_img 100%[===================>]   6.28G  23.5MB/s    in 3m 23s  

2022-05-01 17:18:56 (31.7 MB/s) - ‘data/ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]

--2022-05-01 17:19:27--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving to: ‘map_clsloc.txt’

map_clsloc.txt      100%[====

In [2]:
import tensorflow as tf
import numpy as np

model = tf.keras.applications.ResNet101(weights='imagenet')

mapping_file = open('map_clsloc.txt',mode='r')

validation_file = open('ILSVRC2012_validation_ground_truth.txt',mode='r')

validation_list = validation_file.readlines()
mapping_list_from = mapping_file.readlines()
mapping_list_to = []
for i in range(len(mapping_list_from)):
  mapping_list_to.append(mapping_list_from[i].split(" ")[1])
  mapping_list_from[i] = mapping_list_from[i].split(" ")[0]

for i in range(len(validation_list)):
  validation_list[i] = validation_list[i].split('\n')[0]

179658752/179648224 [==============================] - 1s 0us/step


In [3]:
correct = 0
top_5 = 0

it = 50000

for i in range(it):
  img_path = f'data/ILSVRC2012_val_{(i+1):08d}.JPEG'
  img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = tf.keras.applications.resnet.preprocess_input(x)
  preds = model.predict(x)

  id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=1)[0][0][0]
  index = mapping_list_from.index(id)

  if mapping_list_to[index] == validation_list[i]:
    correct = correct + 1
    top_5 = top_5 + 1
  else:
    for ii in range(1,5):
      id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=5)[0][ii][0]
      index = mapping_list_from.index(id)
      if mapping_list_to[index] == validation_list[i]:
        top_5 = top_5 + 1
        break

  if i!=0 and i%500==0:
    print(str(int(i/500))+'%, Top 1:' + str(correct/i) + ' Top 5:' + str(top_5/i))

49152/35363 [=========================================] - 0s 0us/step
1%, Top 1:0.704 Top 5:0.898
2%, Top 1:0.697 Top 5:0.896
3%, Top 1:0.7086666666666667 Top 5:0.892
4%, Top 1:0.7005 Top 5:0.8885
5%, Top 1:0.7064 Top 5:0.894
6%, Top 1:0.703 Top 5:0.8923333333333333
7%, Top 1:0.6982857142857143 Top 5:0.896
8%, Top 1:0.7 Top 5:0.896
9%, Top 1:0.6997777777777778 Top 5:0.896
10%, Top 1:0.7012 Top 5:0.8952
11%, Top 1:0.7007272727272728 Top 5:0.8947272727272727
12%, Top 1:0.7016666666666667 Top 5:0.8943333333333333
13%, Top 1:0.7024615384615385 Top 5:0.8950769230769231
14%, Top 1:0.7024285714285714 Top 5:0.8944285714285715
15%, Top 1:0.7030666666666666 Top 5:0.8942666666666667
16%, Top 1:0.70375 Top 5:0.89475
17%, Top 1:0.7036470588235294 Top 5:0.8937647058823529
18%, Top 1:0.7024444444444444 Top 5:0.8936666666666667
19%, Top 1:0.702 Top 5:0.8933684210526316
20%, Top 1:0.7014 Top 5:0.8928
21%, Top 1:0.7019047619047619 Top 5:0.8929523809523809
22%, Top 1:0.7015454545454546 Top 5:0.8927272727

In [4]:
accuracy = correct/it*100
top_5_accuracy = top_5/it*100
print('Top 1 Accuracy:'+str(accuracy)+'%')
print('Top 5 Accuracy:'+str(top_5_accuracy)+'%')

Top 1 Accuracy:69.976%
Top 5 Accuracy:89.50200000000001%
